In [ ]:
# Share the sheet for this email:
# reportouvidoria@ouvidoriaitabira.iam.gserviceaccount.com

ID =  '1YVhyxv2jUbDQ-esHHfzgqn_CXSniIP15OxJGQzjkFR8' # Julho
URL = 'https://docs.google.com/spreadsheets/d/{}/edit?usp=sharing'.format(ID)

In [ ]:
titles = {
    'TFD': None,      # TFD 
    'CM': None,       # Central de marcação
    'CEAE': None,     # CEAE
    'UE': None,       # Urgência e emergência
    'CR': None,       # Central de reabilitação
    'SB': None,       # Saúde Bucal
    'Poli': None,     # Policlínica
    'VS': None,       # Vigilância em saúde
    'Assist': None,   # Assistência farmacêutica e insumos
    'ADM': None,      # Administração em saúde 
    'SM': None        # Saúde mental
}

In [ ]:
titles['TFD'] = 'Example.'
titles['Poli'] = ['Example.', 'Example.']

# Login and Sheet load



Create a folder drive that will be used for logo image and fonts.

In [ ]:
# -*- coding: utf-8 -*-
import pandas as pd
import numpy as np

from oauth2client.service_account import ServiceAccountCredentials as Credentials
import gspread # Sheets

In [ ]:
# Login Constants
SCOPE = ['https://www.googleapis.com/auth/spreadsheets.readonly']

# Create an Client

GAUTH = Credentials.from_json_keyfile_name('credentials/nisis_credentials.json',SCOPE)
# Auth the client
GCLIENT = gspread.authorize(GAUTH)

In [ ]:
wb    = GCLIENT.open_by_url(URL) # client open this url
sheet = wb.worksheet('Respostas ao formulário 1')
data = sheet.get_all_values() # get csv titles['TFD'] from spreadsheet

df = pd.DataFrame(data)
df.columns = df.iloc[0] # Nomes das colunas na primeira linha
df = df.iloc[1:] # Os dado começão da segunda linha

df.drop(labels=['OuvidorSUS','Concluído', 'Aguardando Resposta Setor'], 
        axis=1, inplace=True)
df.replace('', np.nan, inplace=True)

df.reset_index(inplace=True)

# Plots

## Plot functions

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from io import StringIO,BytesIO # Ima

sns.set(style='ticks')

In [ ]:
def plot_bar_graph(df, name, graph_title=None,
                   figsize=(13,5),
                   n_step=0,
                   fontsize=14,
                   label_height=.005,
                   x_rotation=0,
                   ann_rotation=0,
                   loc='lower center',
                   bbox_to_anchor=(.5, -.3),
                   legend=False):
    """
    Plot bar graph

    Parameters
    ----------
    df : DataFrame, required
          DataFrame witch the data will be extract.
    name : string, required
          column name inside DataFrame.
    graph_title : string, optional, default: None
          title for the ploted graph.
          if not provided, will be used the column name
    figsize : (float, float), optional, default: (13, 4)
          width, height in inches for the plot.
    n_step : integer, optional, default: 0
          incresse the number of ticks in the Y position
    fontsize: float, optional, default: 14
          size of the smallest font used in the graph
          the title font size will be .5x bigger than fontsize
    label_height: float, optional, default: .005
          height of labels that stand over the bars
    x_rotation: integer, optional, default: 0
    ann_rotation: integer, optional, default: 0
    loc: string or (float, float), optional, default: 'lower center'
          location of the legend
    bbox_to_anchor: (float, float), optional, default: (.5, -.3)
          Box that is used to position the legend in conjunction with loc
  """ 
  
    df2 = df[name].value_counts(normalize=True)

    if not df2.empty:
        fig = plt.figure()
        ax = df2.plot.bar(figsize=figsize)

        graph_title= graph_title if graph_title != None else name.replace(':', '')
        ax.set_title('{}'.format(graph_title), size=fontsize*1.5)

        sum = np.sum([axi.get_height() for axi in ax.patches])

        for p in ax.patches:
            # Percentage of ocurrences plotted for each bar
            ax.annotate(s="{:.1f}%".format(round(p.get_height()*100, 1)),
                        xy=(p.get_x()+p.get_width()*.5,
                            p.get_height()+label_height),
                        horizontalalignment='center',
                        rotation=ann_rotation,
                        size=fontsize)

        yticks = ax.get_yticks()
        step = yticks[1]-yticks[0]
        yticks = ax.set_yticks(np.arange(start=yticks[0], 
                                         stop=yticks[-1]+n_step*step, step=step))
        yticklabels = ax.set_yticklabels(
            ['{:04.1f}%'.format(x*100) for x in ax.get_yticks()], size=fontsize)

        xticklabels = ax.set_xticklabels(labels=ax.get_xticklabels(), 
                                         rotation=x_rotation, size=fontsize)

        total = df[name].value_counts().sum()
        resp = 'resposta' if total==1 else 'respostas'
        if legend:
            legend = ax.legend(['{} ( {} {} )'.format(name.replace(':',''),
                                                    total,resp)],
                    loc=loc,
                    bbox_to_anchor=bbox_to_anchor,
                    framealpha=0,
                    fontsize=fontsize)
        sns.despine()

        fig.tight_layout()
        return fig, total
    else:
        print('A coluna "{}" está vazia'.format(df2.name))
        return None, None
        
        
def func(pct, sum):
    absolute = int(pct/100*sum + 0.3) # we must sum .3 to force correct values
    return "{:.1f}%\n{:d}".format(pct, absolute)

def plot_pie_graph(df, name, graph_title=None,
                   figsize=(13,5),
                   fontsize=14,
                   explode=False,
                   i_explode=0,
                   s_explode=.05,
                   legend_title="",
                   loc="center left",
                   bbox_to_anchor=(1, 0, 0.5, 1),
                   legend=False):
    df2 = df[name].value_counts(normalize=True)

    if not df2.empty:
        fig, ax = plt.subplots(figsize=figsize)

        a_explode = np.array([0.0]*len(df2))
        graph_title= graph_title if graph_title != None else name.replace(':', '')

        if explode:
            a_explode[i_explode] = s_explode

        sum = df[name].value_counts().sum()

        wedges, texts, autotexts = ax.pie(df2,
                                          autopct=lambda pct: func(pct,sum),
                                          explode=a_explode)
        if legend:
            legend = ax.legend(wedges, df2.axes[0],
                    loc=loc,
                    bbox_to_anchor=bbox_to_anchor,
                    title=legend_title,
                    fontsize=fontsize)
          # ax.set_title('{}'.format(graph_title),
          #             size=fontsize*1.5, color='black', weight='bold')

            plt.setp(legend.get_title(), fontsize=fontsize*1.5)
        plt.setp(autotexts, size=fontsize*1.5, color='white', weight="bold")

        ax.axis('equal')
        total = df[name].value_counts().sum()
        resp = 'resposta' if total==1 else 'respostas'
        fig.tight_layout()

        return fig,  total
    else:
        print('A coluna "{}" está vazia'.format(df2.name))
        return None, None

## Generate each plot

In [ ]:
imgBuffer = {}
labelBuffer = {}
imgSizes = {}

class Struct(object):
    def __init__(self,x,y):
        self.x, self.y = x,y
    def getXY(self):
        return (self.x, self.y)

### Tipo de manifestação

In [ ]:
auxImg, auxLbl = False, False
imgSizes['tpManifest'] = Struct(13,10)

auxImg, auxLbl = plot_bar_graph(df=df, 
                                graph_title='', 
                                name='Tipo de Manifestação:',
                                figsize=imgSizes['tpManifest'].getXY())
if auxImg:
    buf = BytesIO()
    auxImg.savefig(buf, format='png')
    buf.seek(0)

imgBuffer['tpManifest'] = buf
labelBuffer['tpManifest'] = auxLbl

### Serviços

In [ ]:
imgSizes['services'] = Struct(11,9)
auxImg, auxLbl = plot_bar_graph(df=df, graph_title='', 
                                name='Serviços:',
                                x_rotation=90,
                                ann_rotation=90,
                                label_height=0.01,
                                n_step=1,
                                bbox_to_anchor=(.5, -.9),
                                figsize=imgSizes['services'].getXY())
if auxImg:
    buf = BytesIO()
    auxImg.savefig(buf, format='png')
    buf.seek(0)

imgBuffer['services'] = buf
labelBuffer['services'] = auxLbl

### Objeto da Manifestação - TFD

In [ ]:
imgSizes['tfd'] = Struct(9,9)
auxImg, auxLbl = plot_bar_graph(df=df, graph_title='', 
                                name='Objeto da Manifestação - TFD',
                                x_rotation=90,
                                ann_rotation=90,
                                label_height=0.03,
                                n_step=2,
                                bbox_to_anchor=(.5, -.8),
                                figsize=imgSizes['tfd'].getXY())
if auxImg:
    buf = BytesIO()
    auxImg.savefig(buf, format='png')
    buf.seek(0)

imgBuffer['tfd'] = buf
labelBuffer['tfd'] = auxLbl

```python
# Para gerar também os outros do obj TFD:
# auxImg, auxLbl = plot_bar_graph(df=df, graph_title='', 
                                name='Objeto da Manifestação - TFD - Outros',
                                x_rotation=90,
                                n_step=2,
                                bbox_to_anchor=(.5, -1.8))
# imgBuffer.append(auxImg)
```

### Objeto de Manifestação - Central de Marcação

In [ ]:
imgSizes['marcacao'] = Struct(15,13)
auxImg, auxLbl = plot_bar_graph(df=df, graph_title='', 
                                name='Objeto da Manifestação - Central de Marcação',
                                ann_rotation=90,
                                x_rotation=90,
                                n_step=1,
                                bbox_to_anchor=(.5, -.75),
                                figsize=imgSizes['marcacao'].getXY())
if auxImg:
    buf = BytesIO()
    auxImg.savefig(buf, format='png')
    buf.seek(0)

imgBuffer['marcacao'] = buf
labelBuffer['marcacao'] = auxLbl

```python
# Caso seja necessário, use o Objeto de Manifestação outros:
# auxImg, auxLbl = plot_bar_graph(df=df, graph_title='', 
                                name='Objeto da Manifestação - CM - Outros',
                                ann_rotation=90,
                                x_rotation=90,
                                n_step=2,
                                bbox_to_anchor=(.5, -1.3))
# imgBuffer.append(auxImg)
```

### Objeto de Manifestação - CEAE

In [ ]:
imgSizes['ceae'] = Struct(9,9)
auxImg, auxLbl = plot_pie_graph(df=df, graph_title='', 
                                name='Objeto da Manifestação - CEAE',
                                fontsize=12, 
                                legend_title='Procedimentos',
                                explode=True,
                                figsize=imgSizes['ceae'].getXY(),
                                legend=True)
if auxImg:    
    buf = BytesIO()
    auxImg.savefig(buf, format='png')
    buf.seek(0)

imgBuffer['ceae'] = buf
labelBuffer['ceae'] = auxLbl

```python
# Ceae outros:
# imgSizes.append(Struct(9,9))
# auxImg, auxLbl = plot_pie_graph(df=df, graph_title='', 
                                name='Objeto da Manifestação - CEAE - Outros',
                                fontsize=10, 
                                explode=True,
                                figsize=imgSizes[-1].getXY(),
                                legend=True)
# buf = BytesIO()
# auxImg.savefig(buf, format='png')
# buf.seek(0)
# imgBuffer.append(buf)

# labelBuffer.append(auxLbl)
```

### Objeto de Manifestação – Urgência e Emergência

In [ ]:
imgSizes['ue'] = Struct(9,9)
auxImg, auxLbl = plot_pie_graph(df=df, name='Objeto da Manifestação - UE',
                                fontsize=10, 
                                explode=True,
                                figsize=imgSizes['ue'].getXY(),
                                legend=True)
if auxImg:
    buf = BytesIO()
    auxImg.savefig(buf, format='png')
    buf.seek(0)

imgBuffer['ue'] = buf
labelBuffer['ue'] = auxLbl

### Objeto de Manifestação – Centro de Reabilitação

In [ ]:
imgSizes['cr'] = Struct(9,9)
auxImg, auxLbl = plot_pie_graph(df=df, name='Objeto da Manifestação - CR',
                                fontsize=10, 
                                explode=True, 
                                figsize=imgSizes['cr'].getXY(),
                                legend=True)
if auxImg:
    buf = BytesIO()
    auxImg.savefig(buf, format='png')
    buf.seek(0)

imgBuffer['cr'] = buf
labelBuffer['cr'] = auxLbl

### Objeto de Manifestação – PSF

In [ ]:
imgSizes['psf'] = Struct(9,9)
auxImg, auxLbl = plot_bar_graph(df=df, graph_title='', 
                                name='Objeto da Manifestação - PSF',
                                x_rotation=90,
                                bbox_to_anchor=(.5, -1), 
                                figsize=imgSizes['psf'].getXY())

if auxImg:    
    buf = BytesIO()
    auxImg.savefig(buf, format='png')
    buf.seek(0)

imgBuffer['psf'] = buf
labelBuffer['psf'] = auxLbl

### Objeto de Manifestação – Saúde Bucal

In [ ]:
imgSizes['bucal'] = Struct(9,9)
auxImg, auxLbl = plot_pie_graph(df=df, name='Objeto da Manifestação - SAÚDE BUCAL',
                                fontsize=10,
                                explode=True, 
                                figsize=imgSizes['bucal'].getXY(),
                                legend=True)
if auxImg:
    buf = BytesIO()
    auxImg.savefig(buf, format='png')
    buf.seek(0)

imgBuffer['bucal'] = buf
labelBuffer['bucal'] = auxLbl

### Objeto de Manifestação – Policlínica

In [ ]:
imgSizes['poli'] = Struct(9,9)
auxImg, auxLbl = plot_pie_graph(df=df, name='Objeto da Manifestação - Policlínica',
                                fontsize=10, 
                                explode=True, 
                                figsize=imgSizes['poli'].getXY(),
                                legend=True)
if auxImg:
    buf = BytesIO()
    auxImg.savefig(buf, format='png')
    buf.seek(0)

imgBuffer['poli'] = buf
labelBuffer['poli'] = auxLbl

### Objeto de Manifestação – Vigilância em Saúde

In [ ]:
imgSizes['vsaude'] = Struct(9,9)
auxImg, auxLbl = plot_pie_graph(df=df, name='Objeto da Manifestação - VIGILÂNCIA EM SAÚDE',
                                fontsize=10, 
                                explode=True, 
                                figsize=imgSizes['vsaude'].getXY(),
                                legend=True)
if auxImg:
    buf = BytesIO()
    auxImg.savefig(buf, format='png')
    buf.seek(0)

imgBuffer['vsaude'] = buf
labelBuffer['vsaude'] = auxLbl

### Objeto de Manifestação – Assistência Farmacêutica, Insumos e Nutrição

In [ ]:
imgSizes['assist'] = Struct(9,9)
auxImg, auxLbl = plot_pie_graph(df=df, name='Objeto da Manifestação - Assistência Farmacêutica, Insumos e Nutrição',
                                fontsize=10, 
                                explode=True, 
                                figsize=imgSizes['assist'].getXY(),
                                legend=True)
if auxImg:
    buf = BytesIO()
    auxImg.savefig(buf, format='png')
    buf.seek(0)

imgBuffer['assist'] = buf
labelBuffer['assist'] = auxLbl

### Objeto de Manifestação – Administração em Saúde

In [ ]:
imgSizes['adm'] = Struct(9,9)
auxImg, auxLbl = plot_pie_graph(df=df, name='Objeto da Manifestação - ADMINISTRAÇÃO EM SAÚDE',
                                fontsize=10, 
                                explode=True, 
                                figsize=imgSizes['adm'].getXY(),
                                legend=True)
if auxImg:
    buf = BytesIO()
    auxImg.savefig(buf, format='png')
    buf.seek(0)

imgBuffer['adm'] = buf
labelBuffer['adm'] = auxLbl

### Objeto de Manifestação – Saúde Mental

In [ ]:
imgSizes['mental'] = Struct(9,9)
auxImg, auxLbl = plot_pie_graph(df=df, name='Objeto da Manifestação - SAÚDE MENTAL',
                                 fontsize=10, 
                                 explode=True, 
                                 figsize=imgSizes['mental'].getXY(),
                                 legend=True)
if auxImg:
    buf = BytesIO()
    auxImg.savefig(buf, format='png')
    buf.seek(0)

imgBuffer['mental'] = buf
labelBuffer['mental'] = auxLbl

### Origem da demanda

In [ ]:
imgSizes['unidade'] = Struct(15,12)
auxImg, auxLbl = plot_bar_graph(df=df, graph_title='', 
                                name='Unidade de Origem:', 
                                n_step=1,
                                label_height=.0025,
                                ann_rotation=90, 
                                x_rotation=90, 
                                bbox_to_anchor=(.5,-.65), 
                                figsize=imgSizes['unidade'].getXY())
if auxImg:
    buf = BytesIO()
    auxImg.savefig(buf, format='png')
    buf.seek(0)

imgBuffer['unidade'] = buf
labelBuffer['unidade'] = auxLbl

### Resolução/Resposta

In [ ]:
imgSizes['resolucao'] = Struct(9,8)
auxImg, auxLbl = plot_bar_graph(df=df, graph_title='', 
                                name='Resolução:', 
                                n_step=1, 
                                figsize=imgSizes['resolucao'].getXY())
if auxImg:
    buf = BytesIO()
    auxImg.savefig(buf, format='png')
    buf.seek(0)

imgBuffer['resolucao'] = buf
labelBuffer['resolucao'] = auxLbl

# PDF

## Imports and mounting settings

In [ ]:
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import A4
from reportlab.lib.styles import ParagraphStyle
from reportlab.lib.units import cm,inch
from reportlab.pdfbase import pdfmetrics
from reportlab.pdfbase.ttfonts import TTFont
from reportlab.platypus import SimpleDocTemplate, XPreformatted, Paragraph, Spacer, PageBreak, Image
from reportlab.lib.enums import TA_LEFT, TA_CENTER, TA_RIGHT, TA_JUSTIFY
from reportlab.lib.fonts import addMapping

## Font family and  paragraphs settings
> The fonts and images are in "nisis.cadastro@gmail.com" drive. Into the folder ouvidoria

In [ ]:
fontPath = 'fonts/arial'

pdfmetrics.registerFont(TTFont('Arial', fontPath +'.ttf'))
pdfmetrics.registerFont(TTFont('Arialb', fontPath + 'bd.ttf'))
pdfmetrics.registerFont(TTFont('Ariali', fontPath + 'i.ttf'))
pdfmetrics.registerFont(TTFont('Arialbi', fontPath + 'bi.ttf'))
# pdfmetrics.registerFont(TTFont('Arialk', fontPath + 'k.ttf'))

pdfmetrics.registerFontFamily(
    'Arial',
    normal='Arial',
    bold='Arialb',
    italic='Ariali',
    boldItalic='Arialbi')

In [ ]:
def getResp(num):
    '''
    Function used in report
    '''
    return 'resposta' if num==1 else 'respostas'

In [ ]:
#page = canvas.Canvas("test.pdf", pagesize=A4)
paragraph_styles = {}
paragraph_styles['Default'] = ParagraphStyle(
                                name='Normal',
                                fontName='Arial',
                                fontSize=12,
                                alignment=TA_JUSTIFY,
                                leading=.6*cm,
                                spaceAfter=.75*cm)

paragraph_styles['DefaultList'] = ParagraphStyle(
                                name='NormalList',
                                parent=paragraph_styles['Default'],
                                leftIndent=2*cm,
                                spaceAfter=.3*cm,
                                spaceBefore=.3*cm,
                                bulletIndent=2*cm)

paragraph_styles['DefaultCentered']= ParagraphStyle(
                                name='NormalC',
                                parent=paragraph_styles['Default'],
                                alignment=TA_CENTER)

paragraph_styles['Centered']= ParagraphStyle(
                                name='NormalCentered',
                                leading=0,
                                parent=paragraph_styles['Default'],
                                alignment=TA_CENTER)

paragraph_styles['Title']= ParagraphStyle(
                                name='Title',
                                parent=paragraph_styles['Centered'],
                                fontSize=14)

## Content

### Date

In [ ]:
# pytz.all_timezones to see all timezones
from pytz import timezone # Defaul timezone it's ahead of Brasil(+)
from datetime import datetime, date

def isMonday():
    return date.today().isoweekday() == 1

# Note that changing linux localtime, doesn't affect Python
BRASIL_TZ = timezone('America/Campo_Grande')
DAY, MONTH, YEAR = datetime.now(BRASIL_TZ).strftime("%d %m %Y").split()

def getMonthName(month, startUpper=False):
    '''
    Need it when execute in different system's locale.

    Parameter
    ---------
    startUpper: (boolean)
    Force to fst letter be upper.

    Return
    ------
    (string) Month's names.
    '''
    month = int(month) - 1 # The rep
    monthName = ['janeiro',
               'fevereiro',
               'março',
               'abril',
               'maio',
               'junho',
               'julho',
               'agosto',
               'setembro',
               'outubro',
               'novembro',
               'dezembro'
    ]
    month = monthName[month-1] if month > 0 else monthName[-1]
    m_up = month[0].upper() + month[1:]

    return month if not startUpper else m_up 


# Get month's name
last_month = getMonthName(int(MONTH), True)
curr_month = getMonthName(int(MONTH)+1, True)

### Functions generic

In [ ]:
def pdf_list(l = None):
    '''
    Create a list with values in list titles['TFD'].
    '''
    if not l:
        print ("None input")
        return None

    parts = []
    parts.append(Paragraph(
        text="Maior incidência (Outros):",
        style=paragraph_styles['Default']
    ))

    if type(l) == str:
        l = [l]

    for i in l:
        parts.append(Paragraph(
            text=i,
            style=paragraph_styles['DefaultList'],
            bulletText='–'
        ))
    return parts

### Base

In [ ]:
parts = []
progName = 'Reportlab - Python PDF'
progVers = '2.0'
fileName = 'Relatório_' + '{} de {} de {}'.format(int(DAY), curr_month, YEAR)

### First page

In [ ]:
month = '{} de {}'.format(last_month, YEAR)
qntManifest = df.shape[0]

parts.append(Paragraph(
    text="<b>OUVIDORIA MUNICIPAL DE SAÚDE DE ITABIRA – MG</b>",
    style=paragraph_styles['Title']))

parts.append(Spacer(1,.7*cm))

parts.append(Paragraph(
    text="RELATÓRIO DE DEMANDAS – DADOS ESTATÍSTICOS",
    style=paragraph_styles['Centered']))

imgLogo = open('img/logo.jpg', 'rb')
parts.append(Image(imgLogo, width=75, height=75, hAlign='CENTER'))

parts.append(Spacer(1,.5*cm))

parts.append(Paragraph(
    text="<b>Período:</b> " + month,
    style=paragraph_styles['Default']))

parts.append(Paragraph(
    text=("Em {} a Ouvidoria recebeu {} manifestações que foram \
          convertidas em demandas. Todas essas demandas foram registradas \
          eletronicamente, tanto através do Ouvidor SUS quanto no sistema \
          de controle interno deste setor e encaminhadas aos setores \
          competentes por intermédio do processo de ouvidoria SUS para \
          receberem o devido tratamento.".format(month, qntManifest)),
    style=paragraph_styles['Default']
))

parts.append(Paragraph(
    text="No Gráfico 1 visualizamos a porcentagem de demandas recebidas \
          por tipos de manifestação.",
    style=paragraph_styles['Default']
))


### Tipo de manifestação

In [ ]:
parts.append(Paragraph(
    text="<b>Tipo de Manifestação</b> ({} {})".format(
        labelBuffer['tpManifest'],getResp(labelBuffer['tpManifest'])),
    style=paragraph_styles['Default']
))
parts.append(Image(imgBuffer['tpManifest'],
                  width= 5 * inch, 
                  height= imgSizes['tpManifest'].y * (5/imgSizes['tpManifest'].x) * inch,
                  kind='proportional',
                  hAlign='CENTER'))
parts.append(Paragraph(
    text="Gráfico 1 – Porcentagem de demandas por tipos de manifestação",
    style=paragraph_styles['DefaultCentered']
))

parts.append(PageBreak())

### Serviços

In [ ]:
parts.append(Paragraph(
    text="O Gráfico 2 demonstra a porcentagem de manifestações por \
          Superitendência e suas repartições.",
    style=paragraph_styles['Default']
))

parts.append(Paragraph(
    text="<b>Serviços</b> ({} {})".format(
        labelBuffer['services'],getResp(labelBuffer['services'])),
    style=paragraph_styles['Default']
))

parts.append(Image(imgBuffer['services'],
                  width= 5.35 * inch, 
                  height= imgSizes['services'].y * (5.35/imgSizes['services'].x) * inch,
                  kind='proportional',
                  hAlign='CENTER'))

parts.append(Paragraph(
    text="Gráfico 2 – Porcentagem de manifestações por Superintendência e \
          suas repartições.",
    style=paragraph_styles['DefaultCentered']
))

parts.append(PageBreak())

### Objeto da Manifestação - TFD

In [ ]:
if labelBuffer['tfd']:
    parts.append(Paragraph(
        text="<b>Objeto de Manifestação – TFD</b> ({} {})".format(
            labelBuffer['tfd'],getResp(labelBuffer['tfd'])),
        style=paragraph_styles['Default']
    ))

    parts.append(Image(imgBuffer['tfd'],
                    width= 5.35 * inch, 
                    height= imgSizes['tfd'].y * (5.35/imgSizes['tfd'].x) * inch,
                    kind='proportional',
                    hAlign='CENTER'))

    if titles['TFD']:
        parts.extend(pdf_list(titles['TFD']))
    parts.append(PageBreak())

### Objeto de Manifestação - Central de Marcação

In [ ]:
if labelBuffer['marcacao']:    
    parts.append(Paragraph(
        text="<b>Objeto de Manifestação – Central de Marcação</b> ({} {})".format(
            labelBuffer['marcacao'],getResp(labelBuffer['marcacao'])),
        style=paragraph_styles['Default']
    ))

    parts.append(Image(imgBuffer['marcacao'],
                    width= 6.5 * inch, 
                    height= imgSizes['marcacao'].y * (6.5/imgSizes['marcacao'].x) * inch,
                    kind='proportional',
                    hAlign='CENTER'))


    if titles['CM']:
        parts.extend(pdf_list(titles['CM']))
    parts.append(PageBreak())

### Objeto de Manifestação - CEAE

In [ ]:
if labelBuffer['ceae']:
    parts.append(Paragraph(
        text="<b>Objeto de Manifestação – CEAE</b> ({} {})".format(
            labelBuffer['ceae'],getResp(labelBuffer['ceae'])),
        style=paragraph_styles['Default']
    ))

    parts.append(Image(imgBuffer['ceae'],
                    width= 5.35 * inch, 
                    height= imgSizes['ceae'].y * (5.35/imgSizes['ceae'].x) * inch,
                    kind='proportional',
                    hAlign='CENTER'))

    if titles['CEAE']:
        parts.extend(pdf_list(titles['CEAE']))
    parts.append(PageBreak())

### Objeto de Manifestação – Urgência e Emergência

In [ ]:
if labelBuffer['ue']:
    parts.append(Paragraph(
        text="<b>Objeto de Manifestação – Urgência e Emergência</b> ({} {})".format(
            labelBuffer['ue'],getResp(labelBuffer['ue'])),
        style=paragraph_styles['Default']
    ))

    parts.append(Image(imgBuffer['ue'],
                    width= 5.35 * inch, 
                    height= imgSizes['ue'].y * (5.35/imgSizes['ue'].x) * inch,
                    kind='proportional',
                    hAlign='CENTER'))

    if titles['UE']:
        parts.extend(pdf_list(titles['UE']))
    parts.append(PageBreak())

### Objeto de Manifestação – Centro de Reabilitação

In [ ]:
if labelBuffer['cr']:
    parts.append(Paragraph(
        text="<b>Objeto de Manifestação – Centro de Reabilitação</b> ({} {})".format(
            labelBuffer['cr'],getResp(labelBuffer['cr'])),
        style=paragraph_styles['Default']
    ))

    parts.append(Image(imgBuffer['cr'],
                    width= 5.35 * inch, 
                    height= imgSizes['cr'].y * (5.35/imgSizes['cr'].x) * inch,
                    kind='proportional',
                    hAlign='CENTER'))

    if titles['CR']:
        parts.extend(pdf_list(titles['CR']))
    parts.append(PageBreak())

### Objeto de Manifestação – PSF

In [ ]:
if labelBuffer['psf']:
    parts.append(Paragraph(
        text="<b>Objeto de Manifestação – PSF</b> ({} {})".format(
            labelBuffer['psf'],getResp(labelBuffer['psf'])),
        style=paragraph_styles['Default']
    ))

    parts.append(Image(imgBuffer['psf'],
                    width= 5.35 * inch, 
                    height= imgSizes['psf'].y * (5.35/imgSizes['psf'].x) * inch,
                    kind='proportional',
                    hAlign='CENTER'))


    if titles['PSF']:
        parts.extend(pdf_list(titles['PSF']))
    parts.append(PageBreak())

### Objeto de Manifestação – Saúde Bucal

In [ ]:
if labelBuffer['bucal']:
    parts.append(Paragraph(
        text="<b>Objeto de Manifestação – Saúde Bucal</b> ({} {})".format(
            labelBuffer['bucal'],getResp(labelBuffer['bucal'])),
        style=paragraph_styles['Default']
    ))

    parts.append(Image(imgBuffer['bucal'],
                    width= 5.35 * inch, 
                    height= imgSizes['bucal'].y * (5.35/imgSizes['bucal'].x) * inch,
                    kind='proportional',
                    hAlign='CENTER'))

    if titles['SB']:
        parts.extend(pdf_list(titles['SB']))
    parts.append(PageBreak())

### Objeto de Manifestação – Policlínica

In [ ]:
if labelBuffer['poli']:
    parts.append(Paragraph(
        text="<b>Objeto de Manifestação – Policlínica</b> ({} {})".format(
            labelBuffer['poli'],getResp(labelBuffer['poli'])),
        style=paragraph_styles['Default']
    ))

    parts.append(Image(imgBuffer['poli'],
                    width= 5.35 * inch, 
                    height= imgSizes['poli'].y * (5.35/imgSizes['poli'].x) * inch,
                    kind='proportional',
                    hAlign='CENTER'))

    if titles['Poli']:
        parts.extend(pdf_list(titles['Poli']))
    parts.append(PageBreak())

### Objeto de Manifestação – Vigilância em Saúde

In [ ]:
if labelBuffer['vsaude']:
    parts.append(Paragraph(
        text="<b>Objeto de Manifestação – Vigilância em Saúde</b> ({} {})".format(
            labelBuffer['vsaude'],getResp(labelBuffer['vsaude'])),
        style=paragraph_styles['Default']
    ))

    parts.append(Image(imgBuffer['vsaude'],
                    width= 5.35 * inch, 
                    height= imgSizes['vsaude'].y * (5.35/imgSizes['vsaude'].x) * inch,
                    kind='proportional',
                    hAlign='CENTER'))

    if titles['VS']:
        parts.extend(pdf_list(titles['VS']))
    parts.append(PageBreak())

### Objeto de Manifestação – Assistência Farmacêutica, Insumos e Nutrição

In [ ]:
if labelBuffer['assist']:
    parts.append(Paragraph(
        text="<b>Objeto de Manifestação – Assistência Farmacêutica, Insumos e Nutrição</b> ({} {})".format(
            labelBuffer['assist'],getResp(labelBuffer['assist'])),
        style=paragraph_styles['Default']
    ))

    parts.append(Image(imgBuffer['assist'],
                    width= 5.35 * inch, 
                    height= imgSizes['assist'].y * (5.35/imgSizes['assist'].x) * inch,
                    kind='proportional',
                    hAlign='CENTER'))

    if titles['Assist']:
        parts.extend(pdf_list(titles['Assist']))
    parts.append(PageBreak())

### Objeto de Manifestação – Administração em Saúde

In [ ]:
if labelBuffer['adm']:
    parts.append(Paragraph(
        text="<b>Objeto de Manifestação – Administração em Saúde</b> ({} {})".format(
            labelBuffer['adm'],getResp(labelBuffer['adm'])),
        style=paragraph_styles['Default']
    ))

    parts.append(Image(imgBuffer['adm'],
                    width= 5.35 * inch, 
                    height= imgSizes['adm'].y * (5.35/imgSizes['adm'].x) * inch,
                    kind='proportional',
                    hAlign='CENTER'))

    if titles['ADM']:
        parts.extend(pdf_list(titles['ADM']))
    parts.append(PageBreak())

### Objeto de Manifestação – Saúde Mental

In [ ]:
if labelBuffer['mental']:
    parts.append(Paragraph(
        text="<b>Objeto de Manifestação – Saúde Mental</b> ({} {})".format(
            labelBuffer['mental'],getResp(labelBuffer['mental'])),
        style=paragraph_styles['Default']
    ))

    parts.append(Image(imgBuffer['mental'],
                    width= 5.35 * inch, 
                    height= imgSizes['mental'].y * (5.35/imgSizes['mental'].x) * inch,
                    kind='proportional',
                    hAlign='CENTER'))

    if titles['SM']:
        parts.extend(pdf_list(titles['SM']))
    parts.append(PageBreak())

### Origem da demanda

In [ ]:
if labelBuffer['unidade']:
    parts.append(Paragraph(
        text="<b>Origem da demanda</b> ({} {})".format(
            labelBuffer['unidade'],getResp(labelBuffer['unidade'])),
        style=paragraph_styles['Default']
    ))

    parts.append(Image(imgBuffer['unidade'],
                    width= 5.35 * inch, 
                    height= imgSizes['unidade'].y * (5.35/imgSizes['unidade'].x) * inch,
                    kind='proportional',
                    hAlign='CENTER'))

    titles['TFD'] = None
    if titles['TFD']:
        parts.extend(pdf_list(titles['TFD']))
    parts.append(PageBreak())

### Resolução/Resposta

In [ ]:
parts.append(Paragraph(
    text="O Gráfico 4 demonstra a porcentagem de manifestações de acordo com \
          a resolução/resposta.",
    style=paragraph_styles['Default']
))
parts.append(Paragraph(
    text="<b>Resolução/Resposta</b> ({} {})".format(
        labelBuffer['resolucao'],getResp(labelBuffer['resolucao'])),
    style=paragraph_styles['Default']
))

parts.append(Image(imgBuffer['resolucao'],
                  width= 6.5 * inch, 
                  height= imgSizes['resolucao'].y * (6.5/imgSizes['resolucao'].x) * inch,
                  kind='proportional',
                  hAlign='CENTER'))

parts.append(Paragraph(
    text="Gráfico 4 – Porcentagem de acordo com a resolução/resposta",
    style=paragraph_styles['Centered']
))


parts.append(PageBreak())

### Saving

In [ ]:
def build():
    global page
    page = SimpleDocTemplate(
        fileName + '.pdf', 
        pagesize=A4,
        topMargin=3*cm,
        leftMargin=3*cm,
        bottomMargin=2*cm,
        rightMargin=2*cm,
        title='Ouvidoria - ' + month,
        author= 'ppcamp and ph')

    page.build(parts)

build()